In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
wavedata=pd.read_csv('data files/townsville-wavedata-1975-2019.csv', low_memory=False)
pd.set_option('display.float_format', lambda x:'%f'%x)

In [ ]:
wavedata['Date/Time']=pd.to_datetime(wavedata['Date/Time'])

In [ ]:
date_time=wavedata['Date/Time']
hs=wavedata['Hs']
hmax=wavedata['Hmax']
tz=wavedata['Tz']
tp=wavedata['Tp']
dir_tp_true=wavedata['Dir_Tp TRUE']
sea_surface_temp=wavedata['SST']

In [ ]:
x=wavedata.drop("Date/Time", axis=1).values
min_max_scaler=preprocessing.MinMaxScaler()
x_scaled=min_max_scaler.fit_transform(x)
normalized_wavedata=pd.DataFrame(x_scaled)